In [20]:
from tensorflow.keras import backend, layers, metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import os
import cv2
import time
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input

import seaborn as sns
import matplotlib.pyplot as plt

import os
import cv2
import numpy as np
from tensorflow.keras.applications.xception import preprocess_input
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Lambda
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.regularizers import l2

In [5]:
def get_encoder(input_shape):
    """ Returns the image encoding model """

    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    print(len(pretrained_model.layers))

    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dropout(0.5),

        layers.BatchNormalization(),
        layers.Dense(256, activation="relu", kernel_regularizer=l2(0.001)),
        layers.Dropout(0.5),

        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")

    print(len(encode_model.layers))
    return encode_model

In [8]:
class DistanceLayer(layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


def get_siamese_network(input_shape = (128, 128, 3)):
    encoder = get_encoder(input_shape)

    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")

    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)

    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
        encoder(anchor_input),
        encoder(positive_input),
        encoder(negative_input)
    )

    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

siamese_network = get_siamese_network()
siamese_network.summary()

133
8
Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Anchor_Input (InputLayer)   [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Positive_Input (InputLayer  [(None, 128, 128, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 Negative_Input (InputLayer  [(None, 128, 128, 3)]        0         []                            
 )                                                                                                
                                                                              

In [9]:
class SiameseModel(Model):
    # Builds a Siamese model based on a base-model
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()

        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # Get the two distances from the network, then compute the triplet loss
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics so the reset_states() can be called automatically.
        return [self.loss_tracker]

In [10]:
siamese_model = SiameseModel(siamese_network)

optimizer = Adam(learning_rate=1e-3, epsilon=1e-01)
siamese_model.compile(optimizer=optimizer)

In [12]:
def extract_encoder(model):
    encoder = get_encoder((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

In [14]:
encoder = extract_encoder(siamese_model)


133
8


In [18]:
# Setting random seeds to enable consistency while testing.
random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)
ROOT = 'dataset'
def read_image(path):
    image_1 = cv2.imread(path)
    image = cv2.resize(image_1, (128, 128))

    if image is not None:
        return image
    else:
        print(f"Error: Unable to load image at path {path}")
        return None

In [11]:
siamese_model.load_weights("Recognition_siamese_model_final")


In [15]:
encoder.load_weights("encoder")


In [27]:


# Function to read and preprocess an image
def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = preprocess_input(image)
    return image

# Function to encode an image using the trained encoder
def encode_image(encoder, image_path):
    image = read_and_preprocess_image(image_path)
    encoded_image = encoder.predict(np.expand_dims(image, axis=0))[0]
    return encoded_image

# Function to get the list of training images
def get_training_images(root_folder):
    training_images = []
    classes = os.listdir(os.path.join(root_folder, 'train'))
    for class_folder in classes:
        class_path = os.path.join(root_folder, 'train', class_folder)
        # for image_file in os.listdir(class_path):
        image1_path = os.path.join(class_path, random.choice(os.listdir(os.path.join(root_folder, 'train', class_folder))))
        training_images.append(image1_path)
        # image2_path = os.path.join(class_path, random.choice(os.listdir(os.path.join(root_folder, 'train', class_folder))))
        # training_images.append(image2_path)

    return training_images, classes


# Root folder containing train, test, and validation folders
root_folder = "dataset/"

# Get the list of training images
training_images, classes = get_training_images(root_folder)

# Encode each training image
encoded_training_images = []

for image_path in training_images:
    encoded_image = encode_image(encoder, image_path)
    encoded_training_images.append(encoded_image)

# Convert the list to a numpy array
encoded_training_images = np.array(encoded_training_images)

# Now, encoded_training_images contains the encoded representations of all training images
# You can use this array for further analysis or classification tasks.


1/1 [==============================] - 0s 40ms/step


In [28]:
def compute_distance(vector1, vector2):
    """
    Compute Euclidean distance between two vectors.
    
    Parameters:
    - vector1, vector2: Numpy arrays representing feature vectors.
    
    Returns:
    - Euclidean distance between vector1 and vector2.
    """
    return np.linalg.norm(vector1 - vector2)

In [36]:


test_dir = 'dataset/test'
test_class = random.choice(os.listdir(test_dir))
test_image = random.choice(os.listdir(os.path.join(test_dir,test_class)))

test_image = preprocess_input(read_image(os.path.join(test_dir, test_class, test_image)))
encoded_test_image = encoder.predict(np.expand_dims(test_image, axis=0))[0]

1/1 [==============================] - 0s 152ms/step


In [45]:
from math import inf


distances_to_classes = []

maximum_dist = inf

for i, encoded_class_image in enumerate(encoded_training_images):
    distance = compute_distance(encoded_test_image, encoded_class_image)
    distances_to_classes.append(distance)

    if classes[i] == test_class:
        print(f"Test Class Distance = {distance}")


    if maximum_dist > distance:
        maximum_dist = distance

print(f"Minimum Distance: {maximum_dist}")

print(len(distances_to_classes))

Test Class Distance = 0.7044041156768799
Minimum Distance: 0.28641846776008606


In [38]:
# predicted_class = classes[np.argmin(distances_to_classes)]


pins_Alvaro Morte
pins_Chris Hemsworth
